In [1]:
# Import Moudles and Packages
import gc
import os
# os.environ["CUDA_LAUNCH_BLOCKING"]="1"
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ['CUDA_VISIBLE_DEVICES']='3'

import time
import random

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["axes.grid"]=False

# Import pytorch modules
import torch
import torch.nn as nn
from torch.utils import data as D

from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.cuda.amp import autocast, GradScaler
from madgrad import MADGRAD

device = 'cuda:2' if torch.cuda.is_available() else 'cpu'

print("| Pytorch version: {}".format(torch.__version__))
print("| GPU: {}".format(torch.cuda.is_available()))
print("| Device : ",device)
print("| Device name: ", torch.cuda.get_device_name(0))
print("| Device count: ", torch.cuda.device_count())


| Pytorch version: 1.10.0+cu102
| GPU: True
| Device :  cuda:1
| Device name:  Tesla T4
| Device count:  4


In [2]:
torch.cuda.empty_cache()
gc.collect()

16

In [3]:
# Import local modules
from src import config as C
from src.models import resnet50d
from src.utils.collate_fn import *
from src.utils.print_overwrite import *
from src.utils.seed import *
from src.dataset import w300_dataset as W

seed_everything(C.SEED)

In [4]:
w_dataset = W.FaceLandmarksDataset(transform=W.Transforms(), type="valid")

len_valid_set = int(0.2*len(w_dataset))
len_train_set = len(w_dataset) - len_valid_set

train_dataset , valid_dataset,  = torch.utils.data.random_split(w_dataset , [len_train_set, len_valid_set])

# shuffle and batch the datasets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=2, shuffle=False, num_workers=4)

train_images, train_landmarks = next(iter(train_loader))
valid_images, valid_landmarks = next(iter(valid_loader))

print(f"| Size of image in train_loader : {train_images.shape}")
print(f"| Size of label in train_loader : {train_landmarks.shape}")
print(f"| Size of image in train_loader : {valid_images.shape}")
print(f"| Size of label in train_loader : {valid_landmarks.shape}")

| Size of image in train_loader : torch.Size([16, 1, 512, 512])
| Size of label in train_loader : torch.Size([16, 27, 2])
| Size of image in train_loader : torch.Size([2, 1, 512, 512])
| Size of label in train_loader : torch.Size([2, 27, 2])


In [5]:
model = resnet.resnet18().cpu()
state_dict = torch.load('/home/ubuntu/workspace/FLD-scratch/result/face_landmarks.pth',map_location='cpu')
model.load_state_dict(state_dict)
model = model.to(device)

start_time = time.time()

with torch.no_grad():
    
    images, landmarks = next(iter(valid_loader))
    
    images = images.to(device)
    landmarks = (landmarks) * 512 #* torch.tensor((512,512))    

    predictions = model(images).cpu()
    predictions = (predictions) * 512 # torch.tensor((512,512))  
    predictions = predictions.view([-1,27,2])
    
    plt.figure(figsize=(10,20))
    
    for img_num in range(2):
        print(images[img_num].shape)
        plt.subplot(2,1,img_num+1)
        plt.imshow(images[img_num].cpu().permute(1,2,0).squeeze(), cmap='gray')
        plt.scatter(predictions[img_num].T[0], predictions[img_num].T[1], c = 'r', s = 5)
        plt.scatter(landmarks[img_num].T[0], landmarks[img_num].T[1], c = 'g', s = 5)

print('Total number of test images: {}'.format(len(valid_dataset)))

end_time = time.time()
print("Elapsed Time : {}".format(end_time - start_time)) 

NameError: name 'resnet' is not defined

In [ ]:
# import timm
# def FaceSynthetics():
#     backbone = timm.create_model("resnet50d",  num_classes=27*2)
#     return backbone

# x = torch.randn([1, 3, 512, 512]).to(device)
# model = FaceSynthetics().to(device)
# out = model(x).to(device)
# print(f"input : {x.shape} | output : {out.size()}")

In [ ]:
# idx = random.randint(0,len(w_dataset))
# image, landmarks = w_dataset[idx]
# landmarks = (landmarks) * 512
# plt.figure(figsize=(5, 5))
# plt.imshow(image.numpy().squeeze(), cmap='gray');
# plt.scatter(landmarks[:,0], landmarks[:,1], s=8);